<a href="https://colab.research.google.com/github/sujasnas/ktcloud_genai/blob/main/103_LLM_%EC%96%B8%EC%96%B4_AI_%EB%A7%9B%EB%B3%B4%EA%B8%B0_%EC%99%84%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **언어 AI 이해**



---



- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.
    - **T4 GPU : 16GB VRAM**

- 💡**코드 내용**
    - 언어 AI의 이해를 돕는 코드
    - 2000년대 이전의 자연어 처리(NLP) 과정 이해를 돕는 예제로 구성함
    - 어떤 모델은 실행시간이 오래 걸릴 수 있음.(코랩에서는 RAM을 모두 사용 후 세션이 다운 될 수 있음)


---

# **[Quicktour] transformers로 확인하는 언어 AI의 기능 예**

- **Hugging Face**
    - Hugging Face는 AI 모델과 데이터셋을 공유·활용할 수 있는 대표적인 오픈소스 플랫폼, 커뮤니티
        - **모델 허브(Model Hub)** : 수만 개의 공개된 머신러닝/딥러닝 모델을 누구나 다운로드·활용 가능
        - **데이터셋 허브(Datasets)** : 다양한 표준/비표준 데이터셋을 간편하게 불러와 실험 가능
        - **Transformers 라이브러리** : BERT, GPT, T5, LLaMA 등 최신 NLP·멀티모달 모델을 손쉽게 사용 가능
        - **PEFT/Accelerate 등 도구** : 파인튜닝·분산 학습·최적화 지원
        - **커뮤니티**: 연구자·개발자가 모델과 코드를 공유하고 협업할 수 있는 생태계

- **transformers**
    - transformers library https://huggingface.co/docs/transformers/index
    - Hugging Face에서 제공하는 사전 훈련된 AI 모델들을 쉽게 사용할 수 있게 해주는 파이썬 라이브러리
    - 복잡한 AI 모델을 간단한 코드로 사용할 수 있게 해주는 도구

- **pipeline**
    - https://huggingface.co/docs/transformers/main/en/main_classes/pipelines
    - pipeline은 자연어 처리(NLP)·비전·오디오 등 다양한 AI 태스크를 쉽게 실행할 수 있도록 미리 구성된 추론(실행) 도구
    - 태스크 이름(예: "sentiment-analysis", "translation")을 입력하면,자동으로 적합한 모델 + 토크나이저 + 전/후처리 로직을 불러와,사용자가 텍스트/이미지/오디오를 바로 넣어 결과를 얻을 수 있게 함

    - **pipeline(sentiment-analysis)**
        - Hugging Face Pipelines Documentation :  
        - 텍스트의 감정(긍정/부정) 분석에 특화된 즉시 사용 가능한 AI 모델을 사용하는 파이프라인의 작업(task) 이름
        - 감정 분석에 특화된 즉시 사용 가능한 AI 모델을 사용하는 작업(task) 이름
        - (AI모델 X) 내부적으로 BERT, RoBERTa, DistilBERT 등의 실제 신경망 모델이 동작

## **예제: 나만의 영화 평론 분석기 (자연어 이해 - NLU)**
- **목표**: 영화 리뷰를 입력하면, 해당 리뷰가 긍정적인지 부정적인지를 맞추고, 리뷰에 언급된 핵심 키워드를 뽑아내는 프로그램
- **핵심 기술**: 감성 분석(Sentiment Analysis)

In [ ]:
!pip show transformers

Name: transformers
Version: 4.56.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.12/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [ ]:
# Hugging Face의 transformers 라이브러리 활용
from transformers import pipeline

# 감성 분석 파이프라인 로드 (sentiment-analysis는 "작업 유형"을 지정하는 것)
sentiment_analyzer = pipeline("sentiment-analysis")  # Task : sentiment-analysis

review = "이 영화 다신 안 봐!"

result = sentiment_analyzer(review)

print('\n✅ 감성 분석(Sentiment Analysis) 결과')
print(f"리뷰: {review}")
print(f"분석 결과: {result}") # [{'label': 'POSITIVE', 'score': 0.99...}]

# 핵심 구문 추출은 별도의 모델이나 기법이 필요
# 예시: "배우들 연기", "시간 가는 줄" 등을 추출

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0



✅ 감성 분석(Sentiment Analysis) 결과
리뷰: 이 영화 다신 안 봐!
분석 결과: [{'label': 'POSITIVE', 'score': 0.9793630838394165}]


### **(AI모델 구성파일) 다운로드 진행바 On/Off**

In [ ]:
import tqdm

# 1. 원래 init을 백업
tqdm_init_backup = tqdm.tqdm.__init__

# # 2. 진행바 끄기
# tqdm.tqdm.__init__ = lambda *args, **kwargs: None

# 3. 다시 켜기 (복원)
tqdm.tqdm.__init__ = tqdm_init_backup


- **(AI모델 구성파일) 파일 구성 예:(Task마다 다를 수 있다)**
|파일 이름	|비유|	핵심 역할|
|---|---|---|
|model.safetensors	|🧠 AI의 뇌 (엔진)|	모델의 학습된 가중치(weights), 즉 실질적인 지능이 담긴 가장 핵심적인 파일입니다.|
|config.json|	📜 AI의 설계도|	모델의 구조(몇 개의 층, 어떤 종류의 모델 등)가 정의된 설정 파일입니다.|
|vocab.txt	|📖 AI의 단어 사전|	모델이 이해할 수 있는 단어(토큰)와 그에 해당하는 고유 번호 목록입니다.|
|tokenizer_config.json	|📝 사전 사용 설명서	|단어 사전을 어떤 규칙(소문자화 여부, 특수 토큰 등)으로 사용할지 정의한 파일입니다.|


- **(AI모델 구성파일) 다운로드된 파일 위치(캐시파일) : 코랩**
    - /root/.cache/huggingface/hub ~ /snapshots/...


In [ ]:
# 캐시 폴더에 어떤 모델들이 저장되어 있는지 목록을 봅니다.
!ls /root/.cache/huggingface/hub/

### **모델 직접 지정하기**

In [ ]:
from transformers import pipeline
import pandas as pd

# 감정 분석 파이프라인 초기화
#   model=<업로더이름>/<모델이름>
#       nlptown → 모델을 공개한 Hugging Face 사용자(혹은 조직) 계정명
#       bert-base-multilingual-uncased-sentiment → 모델의 이름
#           BERT 기반
#           다국어 지원
#           대소문자 구분하지 않음 (uncased)
#           감성 분석용으로 학습된 모델
sentiment_analyzer = pipeline("sentiment-analysis",
                             model="nlptown/bert-base-multilingual-uncased-sentiment" )


# 샘플 리뷰 데이터
reviews = [
    "이 제품 정말 좋아요! 추천합니다.",
    "배송이 너무 늦었어요. 실망스럽네요.",
    "가격 대비 괜찮은 것 같습니다."
]

# 감정 분석 수행
results = []
for review in reviews:
    result = sentiment_analyzer(review)
    results.append({
        'text': review,
        'sentiment': result[0]['label'],
        'confidence': result[0]['score']
    })

# 결과 출력
df = pd.DataFrame(results)

print('\n✅ 감성 분석(Sentiment Analysis) 결과')
print(df)

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0



✅ 감성 분석(Sentiment Analysis) 결과
                   text sentiment  confidence
0   이 제품 정말 좋아요! 추천합니다.   5 stars    0.718316
1  배송이 너무 늦었어요. 실망스럽네요.    1 star    0.425818
2     가격 대비 괜찮은 것 같습니다.   3 stars    0.556177


- **Task별 기본모델 참고** : 버전별로 다를 수 있음

|태스크 이름|기본 모델 예시 (지정하지 않을 때)|
|---|---|
|text-generation | gpt2 |
|text-classification / sentiment-analysis| distilbert-base-uncased-finetuned-sst-2-english |
|fill-mask|bert-base-uncased|
|ner / token-classification|dslim/bert-base-NER 또는 BERT 기반 NER 모델|
|question-answering | distilbert-base-uncased-distilled-squad 또는 BERT 기반 QA 모델|
|summarization | facebook/bart-large-cnn |
|translation (예: translation_en_to_fr) | Helsinki-NLP/opus-mt-en-fr |
|zero-shot-classification  |facebook/bart-large-mnli |
|conversational | microsoft/DialoGPT-small 또는 Chat 모델 |



## **예제: 문장 소설 이어 쓰기 (자연어 생성 - NLG)**
- **목표**: 재미있는 소설의 첫 문장을 제시하면, AI가 그럴듯한 다음 문장들을 자동으로 생성해 하나의 짧은 이야기를 완성하는 프로그램
- **핵심 기술**: 텍스트 생성(Text Generation)

In [ ]:
# Hugging Face의 transformers 라이브러리 활용
from transformers import pipeline

# 텍스트 생성 파이프라인 로드 (예: GPT-2)
text_generator = pipeline("text-generation")

prompt = "어느 날 아침, 잠에서 깨어보니 내 방에 코끼리가 있었다."

story = text_generator(prompt,
                       max_length=100,
                       num_return_sequences=1)

print('\n✅ 텍스트 생성(Text Generation) 결과')
print("--- AI가 완성한 이야기 ---")
print(story[0]['generated_text'])
story

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



✅ 텍스트 생성(Text Generation) 결과
--- AI가 완성한 이야기 ---
어느 날 아침, 잠에서 깨어보니 내 방에 코끼리가 있었다. 저자서로 진드 동, 있었서 국어벌이 아기만 되기리스이 있었다. (3) This is the name of the village where the family lived and where the people were educated. 이리스 써이 조서 전내 가을 내 만서 전 아기만 되기리스이 있었다. (4) The name of the town in which the family lived. 프드목시 전내 가을 내 만서 전내 가을 내 만서 전내 가을 내


[{'generated_text': '어느 날 아침, 잠에서 깨어보니 내 방에 코끼리가 있었다. 저자서로 진드 동, 있었서 국어벌이 아기만 되기리스이 있었다. (3) This is the name of the village where the family lived and where the people were educated. 이리스 써이 조서 전내 가을 내 만서 전 아기만 되기리스이 있었다. (4) The name of the town in which the family lived. 프드목시 전내 가을 내 만서 전내 가을 내 만서 전내 가을 내'}]

In [ ]:
story

[{'generated_text': '어느 날 아침, 잠에서 깨어보니 내 방에 코끼리가 있었다. 저자서로 진드 동, 있었서 국어벌이 아기만 되기리스이 있었다. (3) This is the name of the village where the family lived and where the people were educated. 이리스 써이 조서 전내 가을 내 만서 전 아기만 되기리스이 있었다. (4) The name of the town in which the family lived. 프드목시 전내 가을 내 만서 전내 가을 내 만서 전내 가을 내'}]

## **예제 3: 텍스트 요약 (Text Summarization)**
- **목표**: 긴 글(뉴스 기사, 논문, 보고서 등)을 AI를 이용해 핵심 내용만 담은 짧은 글로 자동 요약하는 프로그램
- **핵심 기술**: 텍스트 요약 (Text Summarization)

In [ ]:
from transformers import pipeline

# 요약 파이프라인 초기화
summarizer = pipeline("summarization",
                     model="facebook/bart-large-cnn")

# 긴 텍스트 예시
long_text = """
인공지능(AI)은 21세기 가장 중요한 기술 중 하나로 여겨지고 있습니다.
특히 자연어 처리 분야에서는 BERT, GPT와 같은 대규모 언어 모델의 등장으로
인해 기계가 인간 수준의 언어 이해와 생성 능력을 보여주고 있습니다.
이러한 발전은 교육, 의료, 금융, 엔터테인먼트 등 다양한 분야에서
혁신적인 변화를 가져오고 있으며, 앞으로도 계속해서 발전할 것으로
예상됩니다. 하지만 동시에 윤리적 문제, 일자리 대체, 편향성 등의
문제도 제기되고 있어 신중한 접근이 필요합니다.
"""

# 요약 수행
summary = summarizer(long_text,
                    max_length=50,
                    min_length=10,
                    do_sample=False)

print('\n✅ 텍스트 요약 (Text Summarization) 결과')
print("원문 길이:", len(long_text.split()))
print("요약문:", summary[0]['summary_text'])
print("요약문 길이:", len(summary[0]['summary_text'].split()))

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0



✅ 텍스트 요약 (Text Summarization) 결과
원문 길이: 62
요약문: Review of BERT, GPT, BERT and GPT-BERT systems. BERT:  PERT: 21    ‘‘  하’   ‘�
요약문 길이: 14


## **예제 4: 제로샷 텍스트 분류 (ZeroShotClassification)**
- **제로샷 텍스트 분류 (ZeroShotClassification)**
    - 정의: 학습 데이터에 없는 새로운 라벨(카테고리)에 대해서도, 사전학습된 언어모델을 활용해 문장을 해당 라벨로 분류하는 방법
    - 원리: 라벨을 “설명 문장(hypothesis)”으로 바꿔서, 자연어 추론(NLI, Natural Language Inference) 문제로 변환 후 모델이 참/거짓을 판단하도록 함.
    - 예:
        - 문장: "이 영화는 정말 재밌었다"
        - 라벨 후보: ["긍정", "부정"]
        - 변환 → "이 문장은 긍정을 표현한다." (참/거짓 예측)
    -
- **목표**: 각 단어와 문장의 의미를 바탕으로 스스로 추론하여 가장 적절한 분류하는 프로그램
- **핵심 기술**: 제로샷 분류(ZeroShotClassification)--> 자연어 추론 (NLI, Natural Language Inference)
    - NLI 모델은 두 문장(전제, 가설)을 보고 그 관계가 '참(Entailment)', '모순(Contradiction)', '중립(Neutral)' 중 무엇인지 판단하도록 훈련됨
    - 각 단어와 문장의 의미를 바탕으로 스스로 추론하여 가장 적절한 분류 진행
- **입력** :
    - 분류할 문장
    - 후보 라벨

In [ ]:
from transformers import pipeline

# 제로샷 분류 파이프라인 생성
classifier = pipeline("zero-shot-classification")

# 분류할 문장
sequence_to_classify = "내일 주주총회에서는 차기 CED 선임 안건이 논의될 예정이다."

# 후보 라벨 (미리 학습시킬 필요 없이, 그냥 원하는 라벨을 나열)
candidate_labels = ['경제', '정치', 'IT', '스포츠', '연예']

# 분류 실행
result = classifier(sequence_to_classify, candidate_labels)

# print 문의 설명을 '제로샷 분류'로 수정했습니다.
print('\n✅ 제로샷 분류 (Zero-Shot Classification) 결과')
print(result)


No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0



✅ 제로샷 분류 (Zero-Shot Classification) 결과
{'sequence': '내일 주주총회에서는 차기 CED 선임 안건이 논의될 예정이다.', 'labels': ['정치', '연예', '경제', '스포츠', 'IT'], 'scores': [0.3559022545814514, 0.2537902593612671, 0.2166021764278412, 0.13524648547172546, 0.038458824157714844]}


In [ ]:
from transformers import pipeline

# 1. XLM-RoBERTa 기반 다국어 제로샷 분류
classifier_xlm = pipeline("zero-shot-classification",
                          model="joeddav/xlm-roberta-large-xnli")

# 2. mDeBERTa-v3 기반 다국어 제로샷 분류
classifier_mdeberta = pipeline("zero-shot-classification",
                               model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

# 3. DeBERTa-v3 기반 영어 전용 제로샷 분류
classifier_deberta = pipeline("zero-shot-classification",
                              model="cross-encoder/nli-deberta-v3-base")

# 테스트 문장
sequence = "내일 주주총회에서는 차기 CEO 선임 안건이 논의될 예정이다."
candidate_labels = ["경제", "정치", "IT", "스포츠", "연예"]

# 실행 예시
print("✅ XLM-RoBERTa 결과")
print(classifier_xlm(sequence, candidate_labels))

print("\n✅ mDeBERTa 결과")
print(classifier_mdeberta(sequence, candidate_labels))

print("\n✅ DeBERTa (영어 위주) 결과")
print(classifier_deberta(sequence, candidate_labels))


config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Device set to use cuda:0


✅ XLM-RoBERTa 결과
{'sequence': '내일 주주총회에서는 차기 CEO 선임 안건이 논의될 예정이다.', 'labels': ['정치', '경제', '연예', 'IT', '스포츠'], 'scores': [0.9451004266738892, 0.023494627326726913, 0.012516791000962257, 0.01149750780314207, 0.0073906355537474155]}

✅ mDeBERTa 결과
{'sequence': '내일 주주총회에서는 차기 CEO 선임 안건이 논의될 예정이다.', 'labels': ['정치', '경제', 'IT', '스포츠', '연예'], 'scores': [0.5334479212760925, 0.1781599074602127, 0.14371581375598907, 0.07297401875257492, 0.07170248031616211]}

✅ DeBERTa (영어 위주) 결과
{'sequence': '내일 주주총회에서는 차기 CEO 선임 안건이 논의될 예정이다.', 'labels': ['IT', '경제', '정치', '연예', '스포츠'], 'scores': [0.28434741497039795, 0.24767908453941345, 0.20274703204631805, 0.1659700572490692, 0.09925633668899536]}


- **[주의!]** 모델을 지정하지 않으면, 기본적으로 영어 데이터 위주로 훈련된 모델이 사용됨
    - 해결 방법: 한국어 모델 + 한국어 가설 템플릿 사용
        - (1) 한국어 성능이 검증된 모델 사용하고,
        - (2) 가설 템플릿 또한 한국어로 명확하게 지정하여 모델이 온전히 한국어 환경에서만 추론하도록 함
    - klue/roberta-large
        - KLUE(Korean Language Understanding Evaluation) 벤치마크는 한국어 모델 성능의 표준

In [ ]:
from transformers import pipeline

# 1. 한국어 성능이 검증된 KLUE 모델을 지정합니다.
# classifier = pipeline("zero-shot-classification",
#                       model="klue/roberta-large")
classifier = pipeline("zero-shot-classification",
                      model="klue/roberta-large")

sequence = "내일 주주총회에서는 차기 임원 선임 안건이 논의될 예정이다."
labels = ['경제', '정치', 'IT', '스포츠', '연예']

# 2. 모델이 추론할 가설의 형태를 한국어로 명확히 지정합니다.
hypothesis_template = "이것은 {} 카테고리의 뉴스이다."   # 이 내용은 {}과/와 관련이 있다.

# 분류 실행 시 템플릿을 함께 전달합니다.
result = classifier(sequence, labels, hypothesis_template=hypothesis_template)

print('\n✅ KLUE 모델과 한국어 템플릿 적용 후 결과')
print(result)

- [추가 정보] 모델과 템플릿 추천 예

In [ ]:

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch

# 방법 1: 한국어 특화 모델들 시도
print("=== 방법 1: 다양한 한국어 모델 테스트 ===")

models_to_try = [
    "klue/roberta-large",
    "beomi/KcELECTRA-base",
    "monologg/kobert",
    "xlm-roberta-large"  # 다국어 모델
]

sequence = "내일 주주총회에서는 차기 임원 선임 안건이 논의될 예정이다."
labels = ['경제', '정치', 'IT', '스포츠', '연예']

# 더 명확한 한국어 템플릿들 시도
templates_to_try = [
    "이 글의 주제는 {}에 관한 내용이다.",
    "이 문장은 {} 분야에 관련된다.",
    "이것은 {} 카테고리의 뉴스이다.",
    "이 내용은 {}과/와 관련이 있다."
]

for model_name in models_to_try:
    try:
        print(f"\n📊 모델: {model_name}")
        classifier = pipeline("zero-shot-classification", model=model_name)

        for i, template in enumerate(templates_to_try):
            try:
                result = classifier(sequence, labels, hypothesis_template=template)
                print(f"템플릿 {i+1}: {template}")
                print(f"1위: {result['labels'][0]} ({result['scores'][0]:.4f})")
                print(f"2위: {result['labels'][1]} ({result['scores'][1]:.4f})")

                if result['labels'][0] == '경제':
                    print("✅ 경제가 1위!")
                print("-" * 50)

            except Exception as e:
                print(f"템플릿 {i+1} 오류: {e}")

    except Exception as e:
        print(f"모델 {model_name} 로드 오류: {e}")

print("\n" + "="*60)

- [추가 정보] 정보 추가해서 키워드 기반 분류 시스템 구축

In [ ]:
# =============================================================================
# 방법 : 키워드 기반 분류 시스템 (가장 실용적)
# =============================================================================

class KoreanTextClassifier:
    def __init__(self):
        # 각 카테고리별 키워드 사전 정의
        self.category_keywords = {
            '경제': [
                # 기업 관련
                '주주총회', 'CEO', '선임', '임명', '사장', '회장', '경영진', '이사회',
                '기업', '회사', '법인', '계열사', '자회사', '모회사',
                # 금융 관련
                '투자', '자금', '예산', '매출', '영업이익', '순이익', '손실', '적자', '흑자',
                '주가', '주식', '상장', 'IPO', '증권', '거래소', '코스피', '코스닥',
                '은행', '금융', '대출', '이자', '환율', '원화', '달러',
                # 사업 관련
                '사업', '업계', '시장', '산업', '제조', '생산', '공급', '판매', '유통',
                '계약', '협약', 'MOU', '인수합병', 'M&A'
            ],
            '정치': [
                '정부', '국회', '대통령', '총리', '장관', '정책', '법안', '개정안',
                '선거', '투표', '후보', '정당', '의원', '국정감사', '정치',
                '행정부', '입법부', '사법부', '지방자치', '시장', '도지사'
            ],
            'IT': [
                '소프트웨어', '하드웨어', '프로그램', '앱', '어플리케이션',
                '인공지능', 'AI', '머신러닝', '딥러닝', '빅데이터', '데이터',
                '컴퓨터', '서버', '클라우드', '네트워크', '인터넷', '웹',
                '스마트폰', '태블릿', '기술', '개발', '프로그래밍', '코딩'
            ],
            '스포츠': [
                '경기', '시합', '경쟁', '선수', '팀', '클럽', '감독', '코치',
                '득점', '골', '승리', '패배', '우승', '준우승', '리그', '토너먼트',
                '올림픽', '월드컵', '아시안게임', '프로야구', '축구', '농구', '배구'
            ],
            '연예': [
                '배우', '가수', '연예인', '아이돌', '스타', '셀러브리티',
                '드라마', '영화', '예능', '방송', 'TV', '라디오',
                '콘서트', '공연', '무대', '앨범', '싱글', '뮤직비디오',
                '엔터테인먼트', '기획사', '데뷔', '컴백'
            ]
        }

    def classify_text(self, text, threshold=0.1):
        """텍스트 분류 실행"""
        scores = {}

        # 각 카테고리별 점수 계산
        for category, keywords in self.category_keywords.items():
            score = 0
            matched_keywords = []

            for keyword in keywords:
                if keyword in text:
                    # 키워드 길이에 따른 가중치 (긴 키워드일수록 더 정확)
                    weight = len(keyword) / 10 + 1
                    score += weight
                    matched_keywords.append(keyword)

            scores[category] = {
                'score': score,
                'matched_keywords': matched_keywords
            }

        # 점수 정규화
        total_score = sum([s['score'] for s in scores.values()])

        if total_score > 0:
            for category in scores:
                scores[category]['normalized_score'] = scores[category]['score'] / total_score
        else:
            # 키워드 매치가 없으면 균등 분배
            for category in scores:
                scores[category]['normalized_score'] = 1 / len(scores)

        # 결과 정렬
        sorted_categories = sorted(scores.items(),
                                 key=lambda x: x[1]['normalized_score'],
                                 reverse=True)

        return {
            'text': text,
            'results': sorted_categories,
            'top_category': sorted_categories[0][0],
            'confidence': sorted_categories[0][1]['normalized_score']
        }

# 분류기 생성 및 테스트
classifier = KoreanTextClassifier()

# 테스트 문장들
test_sentences = [
    "내일 주주총회에서는 차기 CEO 선임 안건이 논의될 예정이다.",
    "국회에서 새로운 법안이 통과되었습니다.",
    "새로운 AI 기술이 개발되어 화제가 되고 있다.",
    "프로야구 시즌이 시작되어 팬들의 관심이 집중되고 있다.",
    "인기 배우가 새로운 드라마에 출연한다고 발표했다."
]

print("\n✅ 키워드 기반 분류 결과:")
for i, sentence in enumerate(test_sentences, 1):
    result = classifier.classify_text(sentence)
    print(f"\n{i}. 문장: {sentence}")
    print(f"   분류: {result['top_category']} (신뢰도: {result['confidence']:.3f})")


    # 상위 3개 결과와 매칭된 키워드 보여주기
    print("   상세 결과:")
    for rank, (category, info) in enumerate(result['results'][:3], 1):
        keywords_str = ', '.join(info['matched_keywords'][:3])
        if keywords_str:
            keywords_str = f" [키워드: {keywords_str}]"
        print(f"     {rank}위: {category} ({info['normalized_score']:.3f}){keywords_str}")




---

